# Trading Strategy Based on Classification
In this notebook, I will try to build a classification model to predict the price movement. First use the Price change to create a binary classification target. Then use the order book data to create features. Finally, use the features to train a classification model.

## 1. Data Preprocessing
We nee to create a new column as the target variable. It means whether the min ask price will go up or down in the next minute. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [26]:
# lob_data= pd.read_csv('E:\\Bristol\\mini_project\\JPMorgan_Set01\\test_datasets\\lob_data_all_converted.csv')
# lob_data

In [101]:
lob_data=pd.read_csv('../datasets/resampled_lob_minALL.csv')
lob_data['Datetime'] = pd.to_datetime(lob_data['Datetime'])
# lob_data.set_index('Datetime', inplace=True)
lob_data

,Datetime,Min Ask Price,Max Bid Price,Total Bid Quantity,Total Ask Quantity,Min Ask Quantity,Max Bid Quantity
0,2025-01-02 00:00:00,271.841538,260.186154,22881.0,16799.0,3,5
1,2025-01-02 00:01:00,262.803138,258.206847,24821.0,12978.0,2,3
2,2025-01-02 00:02:00,267.596615,262.929478,33966.0,11786.0,2,3
3,2025-01-02 00:03:00,267.066274,262.751105,30396.0,15613.0,3,2
4,2025-01-02 00:04:00,268.358289,263.077540,28408.0,16458.0,3,4
...,...,...,...,...,...,...,...
62853,2025-07-01 08:13:00,151.230132,96.942053,18271.0,10399.0,3,4
62854,2025-07-01 08:14:00,200.771852,107.148148,16849.0,10211.0,2,3
62855,2025-07-01 08:15:00,174.894172,107.539877,29053.0,9511.0,2,4
62856,2025-07-01 08:16:00,174.897866,108.716463,33890.0,9471.0,2,6


In [67]:
# lob_data= pd.read_csv('../datasets/resampled_lob_min.csv')
# lob_data['Datetime'] = pd.to_datetime(lob_data['Datetime'])
# 
# lob_data

### Labels
#### 1. Simple Price Change
Price goes up: 1
Price goes down: 0
Only consider the next minute's price change.

In [102]:
# Create the target variable,use the min ask price to calculate the price change
# when the next minute's min ask price is higher than the current minute's min ask price, the label is 1, otherwise, the label is 0.

# ask label
lob_data['ask_label'] = np.where(lob_data['Min Ask Price'].shift(-1) > lob_data['Min Ask Price'], 1, 0)
# bid label
lob_data['bid_label'] = np.where(lob_data['Max Bid Price'].shift(-1) > lob_data['Max Bid Price'], 1, 0)
lob_data

,Datetime,Min Ask Price,Max Bid Price,Total Bid Quantity,Total Ask Quantity,Min Ask Quantity,Max Bid Quantity,ask_label,bid_label
0,2025-01-02 00:00:00,271.841538,260.186154,22881.0,16799.0,3,5,0,0
1,2025-01-02 00:01:00,262.803138,258.206847,24821.0,12978.0,2,3,1,1
2,2025-01-02 00:02:00,267.596615,262.929478,33966.0,11786.0,2,3,0,0
3,2025-01-02 00:03:00,267.066274,262.751105,30396.0,15613.0,3,2,1,1
4,2025-01-02 00:04:00,268.358289,263.077540,28408.0,16458.0,3,4,0,0
...,...,...,...,...,...,...,...,...,...
62853,2025-07-01 08:13:00,151.230132,96.942053,18271.0,10399.0,3,4,1,1
62854,2025-07-01 08:14:00,200.771852,107.148148,16849.0,10211.0,2,3,0,1
62855,2025-07-01 08:15:00,174.894172,107.539877,29053.0,9511.0,2,4,1,1
62856,2025-07-01 08:16:00,174.897866,108.716463,33890.0,9471.0,2,6,1,0


#### 2. Price Change Percentage


In [32]:
# # 定义一个函数来生成多分类标签
# def generate_multiclass_labels(data, future_window=5, small_change_thresh=0.01):
#     """
#     生成多分类标签：
#     - 大幅上升: 2
#     - 小幅上升: 1
#     - 持平: 0
#     - 小幅下降: -1
#     - 大幅下降: -2
#     """
#     # 计算未来一段时间的平均价格
#     future_prices = data['Avg Price'].rolling(window=future_window, min_periods=1).mean().shift(-future_window)
#     
#     # 计算价格变化百分比
#     price_change_percentage = (future_prices - data['Avg Price']) / data['Avg Price']
#     
#     # 生成多分类标签
#     data['Label'] = 0  # 默认持平
#     data.loc[price_change_percentage > small_change_thresh, 'Label'] = 2  # 大幅上升
#     data.loc[(price_change_percentage > 0) & (price_change_percentage <= small_change_thresh), 'Label'] = 1  # 小幅上升
#     data.loc[(price_change_percentage < 0) & (price_change_percentage >= -small_change_thresh), 'Label'] = -1  # 小幅下降
#     data.loc[price_change_percentage < -small_change_thresh, 'Label'] = -2  # 大幅下降
#     
#     # 移除因为滚动计算而产生的NaN值
#     return data.dropna()
# 
# # 应用函数生成标签
# data_labeled = generate_multiclass_labels(data_clean)
# 
# # 查看标签分布情况
# label_distribution = data_labeled['Label'].value_counts()
# 
# label_distribution


NameError: name 'data_clean' is not defined

## 2. Feature Engineering
### 2.1 OFI: Order Flow Imbalance
使用复杂的OFI计算方式（如您的代码所示）相比简单差值有以下优点：
1. **价格敏感性**：考虑了价格的变动，可以更敏感地捕捉市场中的流动性变化。
2. **方向性信息**：不仅仅是数量的差异，还包括了价格变动的方向性，这可以揭示市场的买卖力量的动态。
3. **时间序列分析**：这种方法产生的是时间序列数据，适合进行进一步的时间序列分析，比如用来预测未来的市场走势。

OFI的意义：

1. **价格动力**：如果在一个时间点上OFI为正值，这意味着在那个特定时间段内买单的活动（价格上升或数量增加）超过了卖单的活动，这可能表明买方较为强势，可能会对价格产生上行压力。相反，如果OFI为负值，这可能表明卖方较为强势，可能会对价格产生下行压力。

2. **流动性指标**：OFI还可以反映市场的流动性状态。一个高的正OFI值可能表明流动性被迅速吸收，可能导致价格波动加剧。一个高的负OFI值可能表明流动性充足，但也可能伴随着价格的下跌。

3. **交易决策**：交易员和算法交易策略可以使用OFI来判断是否是进入或退出市场的好时机。比如，在一个OFI持续为正的时段进入市场，可能是基于买方强势的假设。



reference: [Order Flow Imbalance](https://github.com/nicolezattarin/LOB-feature-analysis)

<img src="../imgs/img.png">

where b_m(t) and a_m(t) are respectively bid and ask prices at level m at time t and r_m(t) and q_m(t) are the corresponding volumes.
The OFI is calculated as follows:
$$\( e^m(t_n) = \Delta W^m(t_n) - \Delta V^m(t_n) \)$$


In [103]:
def calculate_ofi(df):
    df['delta_bid'] = df['Max Bid Price'].diff().fillna(0)
    df['delta_ask'] = df['Min Ask Price'].diff().fillna(0)
    
        # 计算 ΔW^m(t_n)
    df['delta_w'] = df.apply(
        lambda row: row['Max Bid Quantity'] if row['delta_bid'] > 0 else
                   (-row['Max Bid Quantity'] if row['delta_bid'] < 0 else
                    row['Max Bid Quantity'] - df.loc[df.index[df.index.get_loc(row.name)-1], 'Max Bid Quantity']
                    if row.name > 0 else 0),
        axis=1)
    
    # 计算 ΔV^m(t_n)
    df['delta_v'] = df.apply(
        lambda row: -row['Min Ask Quantity'] if row['delta_ask'] > 0 else
                   (row['Min Ask Quantity'] if row['delta_ask'] < 0 else
                    row['Min Ask Quantity'] - df.loc[df.index[df.index.get_loc(row.name)-1], 'Min Ask Quantity']
                    if row.name > 0 else 0),
        axis=1)
    
    # 计算 OFI
    df['ofi'] = df['delta_w'] + df['delta_v']
    
    return df['ofi']

calculate_ofi(lob_data)
lob_data=lob_data[['Datetime','Min Ask Price','Max Bid Price','Total Bid Quantity','Total Ask Quantity','Min Ask Quantity','Max Bid Quantity','ofi','ask_label','bid_label']]
# drop the first row (ofi is 0)
lob_data = lob_data.iloc[1:]
lob_data

,Datetime,Min Ask Price,Max Bid Price,Total Bid Quantity,Total Ask Quantity,Min Ask Quantity,Max Bid Quantity,ofi,ask_label,bid_label
1,2025-01-02 00:01:00,262.803138,258.206847,24821.0,12978.0,2,3,-1,1,1
2,2025-01-02 00:02:00,267.596615,262.929478,33966.0,11786.0,2,3,1,0,0
3,2025-01-02 00:03:00,267.066274,262.751105,30396.0,15613.0,3,2,1,1,1
4,2025-01-02 00:04:00,268.358289,263.077540,28408.0,16458.0,3,4,1,0,0
5,2025-01-02 00:05:00,265.378682,260.562412,28895.0,12759.0,2,4,-2,1,0
...,...,...,...,...,...,...,...,...,...,...
62853,2025-07-01 08:13:00,151.230132,96.942053,18271.0,10399.0,3,4,-1,1,1
62854,2025-07-01 08:14:00,200.771852,107.148148,16849.0,10211.0,2,3,1,0,1
62855,2025-07-01 08:15:00,174.894172,107.539877,29053.0,9511.0,2,4,6,1,1
62856,2025-07-01 08:16:00,174.897866,108.716463,33890.0,9471.0,2,6,4,1,0



### 2.2 Time Features
We can get some features from Datetime, such as hour, day of week, etc.

In [104]:
lob_data['Datetime'] = pd.to_datetime(lob_data['Datetime'])
lob_data['Hour'] = lob_data['Datetime'].dt.hour
lob_data['Dayofweek'] = lob_data['Datetime'].dt.dayofweek
lob_data['Day'] = lob_data['Datetime'].dt.day
lob_data['Month'] = lob_data['Datetime'].dt.month

### 2.3 RSI: Relative Strength Index
RSI is a momentum oscillator that measures the speed and change of price movements. The RSI oscillates between zero and 100. Traditionally the RSI is considered overbought when above 70 and oversold when below 30.

In [105]:
def calculate_rsi(data, window=14):
    """
    计算给定数据的相对强弱指数（RSI）。
    
    :param data: 包含价格数据的Pandas Series。
    :param window: 用于计算RSI的窗口大小，默认为14。
    :return: 包含RSI值的Pandas Series。
    """
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# 计算Min Ask Price的RSI
lob_data['RSI'] = calculate_rsi(lob_data['Min Ask Price'], window=14)

### 2.4 Volatility
1. 动态调整交易量
根据波动率的大小来调整每次交易的股票数量或资金比例。波动率越高，市场风险越大，可以减少交易量以降低风险；反之，波动率低时可以增加交易量。

2. 止损和止盈点的调整
使用波动率来动态调整止损点和止盈点。在波动性较大的市场，将止损点设置得更远一些，以避免被市场的正常波动所淘汰；在波动性较小的市场，可以相应地将止损点设置得更近一些。

3. 信号强度和波动率的结合
将波动率作为衡量交易信号强度的一个因素。例如，如果波动率较低而买入信号很强，可能表明一个稳健的上涨趋势；反之，高波动性环境下的交易信号可能需要更谨慎地处理。

In [106]:
# # 设置一个时间窗口
rolling_window = 5  # 例如，我们使用过去5个时间点的数据来计算波动率

# 计算对数收益率
lob_data['Log Return Max Bid'] = np.log(lob_data['Max Bid Price'] / lob_data['Max Bid Price'].shift(1))
lob_data['Log Return Min Ask'] = np.log(lob_data['Min Ask Price'] / lob_data['Min Ask Price'].shift(1))

# 计算滚动标准差作为波动率的度量
lob_data['Volatility Max Bid'] = lob_data['Log Return Max Bid'].rolling(window=rolling_window).std()
lob_data['Volatility Min Ask'] = lob_data['Log Return Min Ask'].rolling(window=rolling_window).std()

# 由于滚动计算会产生缺失值，我们通常会删除这些值
lob_data.dropna(inplace=True)

In [137]:
# Trading volume
def adjust_trade_quantity(volatility, max_tradeable_quantity, base_quantity=1, risk_tolerance=0.9):
    """
    根据波动率和最大可交易量调整交易量。
    volatility: 当前波动率
    max_tradeable_quantity: 该时间点的最大可交易量（对于买入操作，是Min Ask Quantity；对于卖出操作，是Max Bid Quantity）
    base_quantity: 基础交易量
    risk_tolerance: 风险容忍度，较低的值意味着更大的交易量
    """
    # 基于波动率调整的交易量
    # 设置一个缩放因子，以便更好地调整交易量
    scaler_fator = 10
    adjusted_quantity = base_quantity / (volatility * risk_tolerance * scaler_fator)
    adjusted_quantity = max(1, int(adjusted_quantity))  # 确保至少交易1单位，并且是整数

    # 确保交易量不超过最大可交易量
    final_trade_quantity = min(adjusted_quantity, max_tradeable_quantity)

    return final_trade_quantity

## 3. Classification Model
### 3.1 Logistic Classifier
I tested the logistic regression model and I found that if I don't use the ofi feature, the accuracy is only 0.55. But if I use the ofi feature, the accuracy can reach 0.66. So I think the ofi feature is very important for the classification model.

In [108]:
# predict the ask label
# Split the data into train and test
lob_data = lob_data.dropna()
X = lob_data[['Max Bid Price','Min Ask Price','Total Bid Quantity','Total Ask Quantity','Min Ask Quantity','Max Bid Quantity','ofi','RSI','Hour','Dayofweek','Day','Month']]
y = lob_data['ask_label']
# use the last 20% data as test set
split_index = int(len(X)*0.8)
X_train, X_test = X[:split_index], X[split_index:] # [50285, 8],[12572, 8]
y_train, y_test = y[:split_index], y[split_index:]

In [109]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
logistic_model_buy = LogisticRegression()
logistic_model_buy.fit(X_train_scaled, y_train)

# Predict the test set
y_pred = logistic_model_buy.predict(X_test_scaled)

print('Train classification report: \n',classification_report(y_train, logistic_model_buy.predict(X_train_scaled)))
print('Test classification report: \n',classification_report(y_test, y_pred))

Train classification report: 
               precision    recall  f1-score   support

           0       0.69      0.66      0.68     25726
           1       0.66      0.68      0.67     24549

    accuracy                           0.67     50275
   macro avg       0.67      0.67      0.67     50275
weighted avg       0.67      0.67      0.67     50275

Test classification report: 
               precision    recall  f1-score   support

           0       0.81      0.59      0.68      6307
           1       0.67      0.86      0.76      6262

    accuracy                           0.72     12569
   macro avg       0.74      0.72      0.72     12569
weighted avg       0.74      0.72      0.72     12569


In [110]:
# predict the bid label
X = lob_data[['Min Ask Price','Max Bid Price','Total Bid Quantity','Total Ask Quantity','Min Ask Quantity','Max Bid Quantity','ofi','RSI','Hour','Dayofweek','Day','Month']]
y = lob_data['bid_label']
# use the last 20% data as test set
split_index = int(len(X)*0.8)
X_train, X_test = X[:split_index], X[split_index:] # [50285, 8],[12572, 8]
y_train, y_test = y[:split_index], y[split_index:]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
logistic_model_sell = LogisticRegression()
logistic_model_sell.fit(X_train_scaled, y_train)

# Predict the test set
y_pred = logistic_model_sell.predict(X_test_scaled)

print('Train classification report: \n',classification_report(y_train, logistic_model_sell.predict(X_train_scaled)))
print('Test classification report: \n',classification_report(y_test, y_pred))

Train classification report: 
               precision    recall  f1-score   support

           0       0.66      0.65      0.66     25326
           1       0.65      0.66      0.66     24949

    accuracy                           0.66     50275
   macro avg       0.66      0.66      0.66     50275
weighted avg       0.66      0.66      0.66     50275

Test classification report: 
               precision    recall  f1-score   support

           0       0.66      0.71      0.68      6329
           1       0.68      0.63      0.65      6240

    accuracy                           0.67     12569
   macro avg       0.67      0.67      0.67     12569
weighted avg       0.67      0.67      0.67     12569


The result of Logistic model is quite good. The accuracy is 0.62. And the model has a similar performance on the train set and the test set, which means the model is not overfitting.
Now we only get some time features and the ofi feature. We can try to add more features to improve the model.

But we can also try to build a simulator based on the classification model. 

## 4. Trading Simulator
### 4.1 Baseline Strategy
Only use the bid/ask label to trade. If the model predicts the ask price will go up, then buy. If the model predicts the bid price will go down, then sell.

波动率调整: 基于市场波动率来动态调整每次的交易量。在市场波动大时减少交易量，在波动小时增加交易量。

In [140]:
initial_capital = 100000
capital = initial_capital
has_position = False
trade_log = []
capital_history = [initial_capital]
# trade_quantity = 1

for i in range(len(X_test_scaled)):
    current_features = X_test_scaled[i]
    
    # use logistic models to predict the next minute's price change(both ask and bid)
    buy_prediction = logistic_model_buy.predict([current_features])[0]  # 预测下一时刻最小卖出价是否上涨
    sell_prediction = logistic_model_sell.predict([current_features])[0]  # 预测下一时刻最大买入价是否上涨
    
    # use ask price as the buy price, use bid price as the sell price
    buy_price = lob_data['Min Ask Price'].iloc[split_index + i]  # 当前时刻的最小卖出价
    sell_price = lob_data['Max Bid Price'].iloc[split_index + i]  # 当前时刻的最大买入价
    
    # adjust the trade quantity based on the volatility. make the trade quantity no more than the max tradeable quantity
    current_volatility_buy = lob_data['Volatility Min Ask'].iloc[split_index + i] 
    buy_quantity = adjust_trade_quantity(current_volatility_buy,lob_data['Min Ask Quantity'].iloc[split_index + i])
    current_volatility_sell = lob_data['Volatility Max Bid'].iloc[split_index + i]  
    sell_quantity = adjust_trade_quantity(current_volatility_sell,lob_data['Max Bid Quantity'].iloc[split_index + i])
    
    # buy_quantity=lob_data['Min Ask Quantity'].iloc[split_index + i]
    # sell_quantity=lob_data['Max Bid Quantity'].iloc[split_index + i]
    
    # buy policy: if the model predicts the ask price will go up, and the current has no position, then buy
    if buy_prediction and not has_position :

        # 如果模型预测价格上涨且当前没有持仓，则买入
        capital -= buy_price*buy_quantity
        has_position = True
        trade_log.append({'action': 'BUY', 'Time':lob_data['Datetime'].iloc[split_index + i],'price': buy_price, 'capital': capital, 'quantity': buy_quantity,'max quantity':lob_data['Min Ask Quantity'].iloc[split_index + i]})
        
    # sell policy: if the model predicts the bid price will go down, and the current has position, then sell
    elif not sell_prediction and has_position :

        # 如果模型预测价格不上涨且当前有持仓，则卖出
        capital += sell_price*sell_quantity
        has_position = False
        trade_log.append({'action': 'SELL','Time':lob_data['Datetime'].iloc[split_index + i],'price': sell_price, 'capital': capital, 'quantity': sell_quantity,'max quantity':lob_data['Max Bid Quantity'].iloc[split_index + i]})

# 计算最终资本
final_capital = capital
profit = final_capital - initial_capital
number_of_trades = len(trade_log)

# 保留两位小数
print('Final capital: ', round(final_capital, 2))
print('Profit: ', round(profit, 2))
print('Final return: ', round(profit / initial_capital * 100, 2), '%')
print('Number of trades: ', number_of_trades)

Final capital:  477898.26
Profit:  377898.26
Final return:  377.9 %
Number of trades:  6459


In [139]:
trade_log

[{'action': 'BUY',
  'Time': Timestamp('2025-05-28 00:24:00'),
  'price': 189.7460869565217,
  'capital': 99810.25391304347,
  'quantity': 1,
  'max quantity': 3},
 {'action': 'SELL',
  'Time': Timestamp('2025-05-28 00:25:00'),
  'price': 131.3585185185185,
  'capital': 100467.04650563607,
  'quantity': 5,
  'max quantity': 5},
 {'action': 'BUY',
  'Time': Timestamp('2025-05-28 00:26:00'),
  'price': 178.25172413793103,
  'capital': 100288.79478149815,
  'quantity': 1,
  'max quantity': 2},
 {'action': 'SELL',
  'Time': Timestamp('2025-05-28 00:27:00'),
  'price': 128.22666666666666,
  'capital': 100545.24811483148,
  'quantity': 2,
  'max quantity': 3},
 {'action': 'BUY',
  'Time': Timestamp('2025-05-28 00:32:00'),
  'price': 184.0,
  'capital': 100361.24811483148,
  'quantity': 1,
  'max quantity': 2},
 {'action': 'SELL',
  'Time': Timestamp('2025-05-28 00:33:00'),
  'price': 129.70935960591132,
  'capital': 100490.9574744374,
  'quantity': 1,
  'max quantity': 4},
 {'action': 'BUY',